In [ ]:
#! pip install fake_useragent

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd
import random
from fake_useragent import UserAgent
import time

In [2]:
# URL of page to be scraped
country = "au"

job_title_list = []
job_title_index = []
company_list = []
job_id_list = []
location_list = []

# From here we generate a random user agent
ua = UserAgent()

In [8]:
# Get a list of latest proxies from a free website
res = requests.get('https://www.sslproxies.org/', headers={'User-Agent':'Mozilla/5.0'})
soup = BeautifulSoup(res.text,"lxml")
proxy_list = []
for items in soup.select("#proxylisttable tbody tr"):
    proxy_ip = ':'.join([item.text for item in items.select("td")[:2]])
    proxy_list.append(proxy_ip)

print(proxy_list)

['91.187.113.205:53281', '45.33.105.95:8118', '181.48.111.246:49094', '175.100.103.170:55443', '191.242.178.209:3128', '182.16.255.194:43419', '202.148.4.93:8080', '79.101.106.74:60115', '195.98.246.112:80', '200.198.220.159:80', '139.255.199.242:80', '18.183.24.153:80', '180.250.170.210:59778', '181.196.254.202:53281', '51.75.147.35:3128', '84.22.59.202:8080', '102.176.160.70:61279', '177.69.23.177:80', '176.235.182.79:8080', '139.99.102.114:80', '37.220.195.14:53281', '150.107.31.176:8080', '175.106.17.62:57406', '118.27.17.151:3128', '179.60.127.45:32378', '195.239.178.110:48009', '118.174.220.14:43473', '105.30.250.108:53281', '118.27.28.45:3128', '103.69.235.81:8080', '212.191.132.210:8080', '212.73.68.156:3128', '159.224.243.185:37793', '62.23.15.92:3128', '103.36.11.161:14571', '154.72.187.90:55443', '213.6.101.174:23500', '41.79.33.170:8080', '103.87.207.188:48792', '103.87.206.249:48792', '103.125.130.113:8080', '186.10.82.22:59880', '103.241.227.118:6666', '180.183.24.19:3128

In [9]:

# search query for Data Analyst roles
url = f"https://{country}.indeed.com/jobs?q=data+analyst&l="

# Create random parameters
user_agent = ua.random
header = {"user-agent": str(user_agent)}
print(header)

proxy = random.choice(proxy_list)
proxy_protocol = {
    "http"  : proxy,
    "https" : proxy
}
print(proxy)

# Retrieve page with the requests module
response = requests.get(
    url,
    proxies=proxy_protocol,
    headers=header
)

soup = BeautifulSoup(response.text, 'lxml')
results = soup.find('div', id='searchCountPages').text.strip()

print(results)

{'user-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36'}
46.0.203.186:8080
Page 1 of 1,483 jobs


In [37]:
url = f"https://www.indeed.com/jobs?q=data+analyst&start=900"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
results = soup.find('div', id='searchCountPages').text.strip()

In [38]:
print(results)

Page 72 of 16,629 jobs


In [18]:
job_dict = [
    #"Data+Analyst",
    "Data+Scientist",
    "Data+Engineer",
    "Machine+Learning"
]

for i, title in enumerate(job_dict):

    # Reset page to 0
    page = 0
    next_page = True

    while next_page != None:
        url = f"https://{country}.indeed.com/jobs?q={title}&start={page}"
        print("Current page: ", url)

        # Random time gap
        time_gap = random.randrange(3, 7, 1)

        time.sleep(time_gap)
        
        # Retrieve page with the requests module
        response = requests.get(
                            url,
                            #proxies=proxy_protocol,
                            headers=header
                        )
        # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Retrieve the parent divs for all articles
        results = soup.find_all('div', class_='result')
        
        # loop over results to get article data
        for result in results:
            try:
                # scrape the article header 
                job_title = result.find('a', class_='jobtitle').text.strip()
                job_index = i+2
                company = result.find('span', class_='company').text.strip()
                job_id = result.get('id')
                location = result.find(class_='location').text
                
                job_title_list.append(job_title)
                job_title_index.append(job_index)
                company_list.append(company)
                location_list.append(location)
                job_id_list.append(job_id)
                
            except:
                pass
        
        page += 10
        next_page = soup.find('span', class_ ='np')

        # Every 10 pages, get random UA
        if page % 100 == 0:
            user_agent = ua.random
            header = {"user-agent": str(user_agent)}
            print(f"---------------\n\
                A new user-agent was created:\n\
                {user_agent}\n----------------")
        
print("===================\nScraping completed")

Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=0
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=10
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=20
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=30
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=40
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=50
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=60
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=70
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=80
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=90
---------------
                A new user-agent was created:
                Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1464.0 Safari/537.36
----------------
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=100
Current page:  https://au.indeed.com/jobs?q=Data+Analyst&start=110
Current pag

KeyboardInterrupt: 

In [27]:
next_page = soup.find('span', class_ ='np')
next_page

<span class="np"><svg fill="none" height="24" width="24"><path d="M15.41 7.41L14 6l-6 6 6 6 1.41-1.41L10.83 12l4.58-4.59z" fill="#2D2D2D"></path></svg></span>

In [19]:
#putting list into dataframe

AU_jobmarket = {"Job ID" : job_id_list,
                "Job Title Index" : job_title_index,
                "Job Title" : job_title_list, 
                "Company Name" : company_list, 
                "Company Location" : location_list}

In [21]:
AU_jobmarket_df = pd.DataFrame(AU_jobmarket)
AU_jobmarket_df.tail(30)

,Job ID,Job Title Index,Job Title,Company Name,Company Location
2443,p_1c98ecb647b58332,1,Customer Experience Data Insights Analyst,Prudential Investment Company of Australia,Sydney NSW
2444,p_fbc82da9c7e10bf7,1,Spatial Data Analyst (Graduate role) - Intervi...,Ever Better Recruitment (EBR),Sydney NSW
2445,p_d9c7db2de9c60874,1,New graduate job: Spatial Data Analyst,Ever Better Recruitment (EBR),Sydney NSW
2446,p_b48c8a3a04111db1,1,Data Analyst,Talenza,Sydney NSW
2447,p_61c49d9dd42b891c,1,Data / Insights Analyst,Sustainability Consulting,Sydney NSW
2448,p_047e0c364d5dcf3d,1,"C#, .NET/SQL Data Analyst/Programmer",CYOS Solutions,Canberra ACT
2449,p_2390292684ffd535,1,Jr Product Analyst,HelloFresh,Sydney NSW
2450,p_feebd756398ae0d0,1,Data Migration Analyst,FourQuarters,Dingley VIC
2451,p_8dc2ab4b4b44ef85,1,Data Analyst,Foundation Recruitment,Brisbane QLD
2452,p_6a1c5895a1390e5b,1,Business Analyst,OHC Guide,Newcastle NSW


In [ ]:
AU_jobmarket_df.to_csv("../Clean Data/AU-JobMarket.csv")

In [26]:
a = set(job_id_list)
len(a)

846